In [1]:
import os

In [33]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [34]:

GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")

In [35]:
LANGCHAIN_PROJECT

'pr-abandoned-cluster-80'

In [7]:

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [8]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

In [9]:
'''from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")'''
from langchain_groq import ChatGroq
import os
llm=ChatGroq(model_name="Gemma2-9b-It")

# Simple AI Assistant

In [10]:
while True:
    question=input("type your question. if you want to quit the chat write quit")
    if question !="quit":
        print(llm.invoke(question).content)
    else:
        print("goodbye take care yourself")
        break

KeyboardInterrupt: Interrupted by user

In [11]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [12]:
store={}

In [13]:

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [14]:
config = {"configurable": {"session_id": "firstchat"}}

In [16]:
model_with_memory=RunnableWithMessageHistory(llm,get_session_history)

In [17]:
model_with_memory.invoke(("Hi! I'm Priyal"),config=config).content

"Hi Priyal, it's nice to meet you! 👋\n\nWhat can I do for you today?\n"

In [18]:
model_with_memory.invoke(("tell me what is my name?"),config=config).content

'Your name is Priyal, as you told me at the beginning! 😊  \n'

In [19]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Priyal", additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Priyal, it's nice to meet you! 👋\n\nWhat can I do for you today?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 16, 'total_tokens': 41, 'completion_time': 0.045454545, 'prompt_time': 0.001930486, 'queue_time': 0.15281932399999998, 'total_time': 0.047385031}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-67543ba2-d14f-4b1c-8682-608329c07d5b-0', usage_metadata={'input_tokens': 16, 'output_tokens': 25, 'total_tokens': 41}), HumanMessage(content='tell me what is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Priyal, as you told me at the beginning! 😊  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 56, 'total_to

# RAG with LCEL

In [23]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [22]:
!pip install langchain-community

  Using cached langchain_community-0.3.20-py3-none-any.whl.metadata (2.4 kB)
  Using cached aiohttp-3.11.14-cp313-cp313-win_amd64.whl.metadata (8.0 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.8.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached numpy-2.2.4-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached frozenlist-1.5.0-cp313-cp313-win_amd64.whl.metadata (14 kB)
  Using cached multidict-6.2.0-cp313-cp313-win_amd64.whl.metadata (5.1 kB)
  Using cached yarl-1.18.3-cp313-cp313-win_amd64.whl.metadata (71 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
 


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
### Reading the txt files from source directory

loader = DirectoryLoader('C:\\Users\\priya\\Downloads\\test\\data', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()


### Creating Chunks using RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

###  BGE Embddings

'''from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)
'''

### Creating Retriever using Vector DB

db = Chroma.from_documents(new_docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 4})

In [28]:
!pip install chromadb

  Using cached chromadb-0.6.3-py3-none-any.whl.metadata (6.8 kB)
  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached chroma_hnswlib-0.7.6-cp313-cp313-win_amd64.whl
  Using cached fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
  Using cached uvicorn-0.34.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached posthog-3.21.0-py2.py3-none-any.whl.metadata (2.9 kB)
  Using cached onnxruntime-1.21.0-cp313-cp313-win_amd64.whl.metadata (4.9 kB)
  Using cached opentelemetry_api-1.31.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.31.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached opentelemetry_instrumentation_fastapi-0.52b1-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemetry_sdk-1.31.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached pypika-0.48.9-py2.py3-none-any.whl
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)


In [31]:

retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    )

In [32]:
question ="what is llama3? can you highlight 3 important points?"
print(retrieval_chain.invoke(question))

Based on the provided text, here are three important points about Llama 3:

1. **Parameter Count:** Llama 3 has an 8B parameter version. 
2. **Release Date:** Llama 3 was released in April 2024.
3. **Accessibility:** Different versions of Llama 3 have been made accessible outside of Meta's internal services. 


Let me know if you'd like me to elaborate on any of these points! 



# Let's Start with Tools and Agents

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [ ]:
api_wrapper=WikipediaAPIWrapper()

In [ ]:
tool=WikipediaQueryRun(api_wrapper=api_wrapper)

In [ ]:
tool.name

'wikipedia'

In [ ]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [ ]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [ ]:
print(tool.run({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.



Page: Retrieval-augmented generation
Summary: Retrieval augmented generation (RAG) is a type of generative artificial intelligence that has information retrieval capabilities. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information in preference to information drawn from its own vast, static training data. This allows LLMs to use domain-specific and/or updated information.  
Use cases include providing chatbot access to internal company data, or giving factual information only from an authoritative source.



Page: DataStax
Sum

In [ ]:
from langchain_community.tools import YouTubeSearchTool

In [ ]:
tool2=YouTubeSearchTool()

In [ ]:
tool2.name

'youtube_search'

In [ ]:
tool2.run("sunny savita")

"['https://www.youtube.com/watch?v=V9tJCQoBakA&pp=ygUMc3Vubnkgc2F2aXRh', 'https://www.youtube.com/watch?v=6GnsndNL5u4&pp=ygUMc3Vubnkgc2F2aXRh']"

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [ ]:
tool3=TavilySearchResults()

In [ ]:
tool.invoke({"query": "What happened in the latest burning man floods"})

'Page: Burning Man\nSummary: Burning Man is a week-long large-scale desert event focused on "community, art, self-expression, and self-reliance" held annually in the western United States. The event\'s name comes from its culminating ceremony: the symbolic burning of a large wooden effigy, referred to as the Man, that occurs on the penultimate night, the Saturday evening before Labor Day. Since 1991, the event has been at Black Rock City in northwestern Nevada, a temporary city erected in the Black Rock Desert about 100 miles (160 km) north-northeast of Reno. According to Burning Man co-founder Larry Harvey in 2004, the event is guided by ten stated principles: radical inclusion, gifting, decommodification, radical self-reliance, radical self-expression, communal effort, civic responsibility, leaving no trace, participation, and immediacy.\nBurning Man features no headliners or scheduled performers; instead, participants create all the art, activities, and events. Artwork includes expe

In [ ]:
from langchain.agents import AgentType
from langchain.agents import load_tools 
from langchain.agents import initialize_agent

In [ ]:
tool=load_tools(["wikipedia"],llm=llm)

In [ ]:
agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

In [ ]:
agent.run("What is current GDP of India?")



> Entering new AgentExecutor chain...
Action: [wikipedia]
Action Input: GDP of india
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Action: [wikipedia]
Action Input: GDP of India
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Action: [wikipedia]
Action Input: India GDP
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Action: [wikipedia]
Action Input: GDP of India
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Action: [wikipedia]
Action Input: India GDP
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Action: [wikipedia]
Action Input: GDP of India
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Action: [wikipedia]
Action Input: GDP of India
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Action: [wikipedia]
Action Input: India GDP
Observation: [wikipedia] is not a valid tool, t

'Agent stopped due to iteration limit or time limit.'